In [362]:
# Import all the required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error
import seaborn as sns
from sklearn.metrics import accuracy_score
import os
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [363]:
# Get current directory
dirpath = os.getcwd()
print("current directory is : " + dirpath)

current directory is : /content/gdrive/My Drive/Colab Notebooks/Bike_Rental_Count/BikeRentalProject1


In [364]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [365]:
# Get file path
# day_file = "/home/someshugar/Project 1/day.csv"
%cd /content/gdrive/My Drive/Colab Notebooks/Bike_Rental_Count/BikeRentalProject1

/content/gdrive/My Drive/Colab Notebooks/Bike_Rental_Count/BikeRentalProject1


In [366]:
!ls

 bike_rental.R	 Figure3.png   Figure9.png
 DataN0103.pdf	 Figure4.png   prediction.csv
 day.csv	 Figure5.png  'Project1 Bike Renting -Final model.ipynb'
 Figure10.png	 Figure6.png  'Project1_Bike Renting New Report.pdf'
 Figure1.png	 Figure7.png   sample-report.pdf
 Figure2.png	 Figure8.png


In [367]:
# Read file  using pandas and check shape
df = pd.read_csv("day.csv", index_col=0)
print(df.shape)

(731, 15)


In [368]:
# View and analyse head of the Dataframe
df.head()

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
instant,,,,,,,,,,,,,,,
1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [369]:
# Date time conversion
df.dteday = pd.to_datetime(df.dteday, format='%Y-%m-%d')

In [370]:
df['hr'] = df['dteday'].dt.hour

In [371]:
# df['day'] = df['dteday'].dt.day
df.drop('dteday',axis=1,inplace=True)

In [372]:
df.dtypes

season          int64
yr              int64
mnth            int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
hr              int64
dtype: object

In [373]:
# Find columns with NaN
df.isnull().sum(axis=0)

season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
hr            0
dtype: int64

In [374]:

# categorical variables
df['season'] = df.season.astype('category')
df['holiday'] = df.holiday.astype('category')
df['weekday'] = df.weekday.astype('category')
df['weathersit'] = df.weathersit.astype('category')
df['workingday'] = df.workingday.astype('category')
df['mnth'] = df.mnth.astype('category')
df['yr'] = df.yr.astype('category')
df['hr'] = df.hr.astype('category')

In [375]:
df.drop(['casual','registered'],axis=1,inplace=True)

In [376]:
#Feature Engineering
# Defining categorical variables encoder method
def fit_transform_ohe(df,col_name):
    # label encode the column
    le = preprocessing.LabelEncoder()
    le_labels = le.fit_transform(df[col_name])
    df[col_name+'_label'] = le_labels
    # one hot encoding
    ohe = preprocessing.OneHotEncoder()
    feature_arr = ohe.fit_transform(df[[col_name+'_label']]).toarray()
    feature_labels = [col_name+'_'+str(cls_label) for cls_label in le.classes_]
    features_df = pd.DataFrame(feature_arr, columns=feature_labels)
    return le,ohe,features_df

# given label encoder and one hot encoder objects, 
# encode attribute to ohe
def transform_ohe(df,le,ohe,col_name):
    # label encode
    col_labels = le.transform(df[col_name])
    df[col_name+'_label'] = col_labels
    
    # ohe 
    feature_arr = ohe.fit_transform(df[[col_name+'_label']]).toarray()
    feature_labels = [col_name+'_'+str(cls_label) for cls_label in le.classes_]
    features_df = pd.DataFrame(feature_arr, columns=feature_labels)
    
    return features_df
# Divide the dataset into training and testing sets
X, X_test, y, y_test = train_test_split(df.drop('cnt',axis=1),
                                        df.iloc[:,-1],
                                        test_size=0.33,
                                        random_state=42)
X.reset_index(inplace=True)
y = y.reset_index()

X_test.reset_index(inplace=True)
y_test = y_test.reset_index()
# Encoding all the categorical features
cat_attr_list = ['season','holiday',
                 'weathersit','workingday',
                 'hr','weekday','mnth','yr']
# though we have transformed all categoricals into their one-hot encodings, note that ordinal
# attributes such as hour, weekday, and so on do not require such encoding.
numeric_feature_cols = ['temp','hum','windspeed',
                        'hr','weekday','mnth','yr']
subset_cat_features =  ['season','holiday','weathersit','workingday']

###############
# Train dataset
###############
encoded_attr_list = []
for col in cat_attr_list:
    return_obj = fit_transform_ohe(X,col)
    encoded_attr_list.append({'label_enc':return_obj[0],
                              'ohe_enc':return_obj[1],
                              'feature_df':return_obj[2],
                              'col_name':col})


feature_df_list  = [X[numeric_feature_cols]]
feature_df_list.extend([enc['feature_df'] \
                        for enc in encoded_attr_list \
                        if enc['col_name'] in subset_cat_features])

train_df_new = pd.concat(feature_df_list, axis=1)
print("Train dataset shape::{}".format(train_df_new.shape))

##############
# Test dataset
##############
test_encoded_attr_list = []
for enc in encoded_attr_list:
    col_name = enc['col_name']
    le = enc['label_enc']
    ohe = enc['ohe_enc']
    test_encoded_attr_list.append({'feature_df':transform_ohe(X_test,
                                                              le,ohe,
                                                              col_name),
                                   'col_name':col_name})
    
    
test_feature_df_list = [X_test[numeric_feature_cols]]
test_feature_df_list.extend([enc['feature_df'] \
                             for enc in test_encoded_attr_list \
                             if enc['col_name'] in subset_cat_features])

test_df_new = pd.concat(test_feature_df_list, axis=1) 
print("Test dataset shape::{}".format(test_df_new.shape))

Train dataset shape::(489, 18)
Test dataset shape::(242, 18)


In [377]:
test_df_new.columns

Index(['temp', 'hum', 'windspeed', 'hr', 'weekday', 'mnth', 'yr', 'season_1',
       'season_2', 'season_3', 'season_4', 'holiday_0', 'holiday_1',
       'weathersit_1', 'weathersit_2', 'weathersit_3', 'workingday_0',
       'workingday_1'],
      dtype='object')

In [378]:
train_df_new.columns

Index(['temp', 'hum', 'windspeed', 'hr', 'weekday', 'mnth', 'yr', 'season_1',
       'season_2', 'season_3', 'season_4', 'holiday_0', 'holiday_1',
       'weathersit_1', 'weathersit_2', 'weathersit_3', 'workingday_0',
       'workingday_1'],
      dtype='object')

In [379]:
# Constructing train dataset
X_train = train_df_new
y_train= y

# Constructing test dataset
X_test = test_df_new
y_test = y_test
print(X.shape,y.shape)

(489, 21) (489, 2)


# LinearRegression Model

In [380]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


In [381]:
rmse = sqrt(mean_squared_error(y_test, y_pred))
print("RMSE =", rmse)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))


RMSE = 6.452570501009242
Variance score: 1.00


In [382]:
# The coefficients
print('Coefficients: \n', model.coef_)



Coefficients: 
 [[-1.76553880e+00  7.45618934e+00 -3.93785062e-02 -8.88178420e-15
   1.26467218e-01  3.04467031e+01  3.66027914e+02  7.12080922e-01
  -1.65096817e-01  3.54829952e-02 -5.82467100e-01 -7.23083102e-03
   7.23083102e-03  9.37443526e-01 -8.97895960e-01 -3.95475660e-02
  -2.15261926e-01  2.15261926e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [383]:
from sklearn.metrics import mean_squared_log_error
def rmsle(y_test,predictions):
    return np.sqrt(mean_squared_log_error( y_test, predictions ))

# RandomForestRegressor Model

In [384]:
from sklearn.ensemble import RandomForestRegressor
RFmodel = RandomForestRegressor()

In [385]:
RFmodel.fit(X_train, y_train)
RF_y_pred = RFmodel.predict(X_test)



In [386]:

rmse = sqrt(mean_squared_error(y_test, RF_y_pred))
print("RMSE =",rmse)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, RF_y_pred))
print("RMSLE: ",np.sqrt(mean_squared_log_error( y_test, RF_y_pred )))

RMSE = 5.539526339509155
Variance score: 1.00
RMSLE:  0.1004307679302777


# DecisionTreeRegressor Model

In [387]:
DT_model = DecisionTreeRegressor()
DT_model.fit(X_train, y_train)
DT_y_pred = DT_model.predict(X_test)   

In [388]:
rmse = sqrt(mean_squared_error(y_test, DT_y_pred))
print("RMSE =", rmse)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, DT_y_pred))
print("RMSLE: ",np.sqrt(mean_squared_log_error( y_test, DT_y_pred )))

RMSE = 6.702965915631356
Variance score: 1.00
RMSLE:  0.1518109461669195
